## This notebook creates CSVs with summary statistics for lethal and non-lethal sampling
The table shows a comparison of lethal and non-lethal sampling.
Non-lethal sampling is source="Salmon Coast Field Station / Raincoast Research" and 'Marty Krkosek'
The data is constrained to
 - zone 3.3,
 - years 2008 and 2009,
 - out-migration months (March-June)
 - Chum and pink salmon

This script produces four CSVs: (non-lethal, lethal) X (2008, 2009)

In [1]:
from pathlib import Path
import pandas as pd

In [2]:
# define data location constants
DATA_DIR = Path('..') / 'source_data'
WILD_FISH_DATA = DATA_DIR / 'all_wild_fish_lice.csv'
WILD_EVENT_DATA = DATA_DIR / 'all_wild_sample_events.csv'

OUTPUT_DIR = Path('..') / 'output' / 'Table_7'

In [3]:
# import and merge the data
events_df = pd.read_csv(WILD_EVENT_DATA, parse_dates=['sampledate'])
fish_df = pd.read_csv(WILD_FISH_DATA)
wild_df = pd.merge(events_df, fish_df, on='event_id', how='left')

In [4]:
# create month and year columns
wild_df['year'] = wild_df.sampledate.dt.year
wild_df['month'] = wild_df.sampledate.dt.month

In [5]:
# make a new column for count of all lice
wild_df['all_lice'] = wild_df.loc[:, 'lep_cop':'unknown_unknown'].sum(axis=1)

In [6]:
# constrain to the data that we are interested in
wild_df = wild_df[(wild_df.dfozone == "3_3")]
wild_df = wild_df[(wild_df.year.isin([2008, 2009]))]
wild_df = wild_df[(wild_df.month.isin([3, 4, 5, 6, 7]))]
wild_df = wild_df[(wild_df.fish_species.isin(['Chum Salmon', 'Pink Salmon']))]
wild_df.head()

,event_id,sampledate,region,dfozone,sample_site,latitude,longitude,source,fish_id,length,...,cal_chal,cal_motile,cal_unknown,unknown_cop,unknown_chal,unknown_motile,unknown_unknown,year,month,all_lice
1868,130.0,2008-04-28,Broughton Archipelago,3_3,Alder Point,50.869999,-126.87,Fisheries and Oceans Canada,37358.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,4,0.0
1869,130.0,2008-04-28,Broughton Archipelago,3_3,Alder Point,50.869999,-126.87,Fisheries and Oceans Canada,37359.0,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,4,0.0
1870,130.0,2008-04-28,Broughton Archipelago,3_3,Alder Point,50.869999,-126.87,Fisheries and Oceans Canada,37360.0,37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,4,0.0
1871,130.0,2008-04-28,Broughton Archipelago,3_3,Alder Point,50.869999,-126.87,Fisheries and Oceans Canada,37361.0,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,4,0.0
1872,130.0,2008-04-28,Broughton Archipelago,3_3,Alder Point,50.869999,-126.87,Fisheries and Oceans Canada,37362.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008,4,0.0


In [7]:
wild_df.source.unique()

array(['Fisheries and Oceans Canada', 'Marty Krkosek',
       'Salmon Coast Field Station / Raincoast Research'], dtype=object)

In [8]:
# label the non-lethal sampling
NONLETHAL = 'nonlethal'
LETHAL = 'lethal'
# All sampling from Marty Krkosek and Salmon Coast are non-lethal
nonlethal_sources = ['Marty Krkosek', 'Salmon Coast Field Station / Raincoast Research']

wild_df['sampling'] = wild_df.source.apply(lambda x: NONLETHAL if x in nonlethal_sources else LETHAL)

In [9]:
def fill_year_stats(input_df, lethality_type):
    """
    Creates a dataframe from the input data. Output is number of fish, non-infested fish count, infested fish count, lice count,
    prevalence, intensity. Output is broken down by month.
    :param input_df: restricted to lethal or non-lethal and year
    :type input_df: pandas.Dataframe object
    :param lethality_type: Indicator of type of data, will be used as first column name
    :type lethality_type: str
    :return: Dataframe of summary stats as per table 7
    :rtype: pandas.Dataframe object
    """
    month_list = sorted(list(input_df.month.unique()))

    output_dict = {lethality_type: []}
    for month in month_list:
        output_dict[month] = []

    # keep track of some numbers to help later calculations
    num_all_fish = {}
    num_infested_fish = {}
    num_all_lice = {}

    # get counts of all fish
    output_dict[lethality_type].append('All fish')
    for month in month_list:
        num_fish = len(input_df[input_df.month == month])
        output_dict[month].append(num_fish)

        num_all_fish[month] = num_fish

    # get counts of non-infested fish
    output_dict[lethality_type].append('Non-infested')
    for month in month_list:
        num_fish = len(input_df[(input_df.month == month) & (input_df.all_lice == 0)])
        output_dict[month].append(num_fish)

    # get counts of infested fish
    output_dict[lethality_type].append('Infested')
    for month in month_list:
        num_fish = len(input_df[(input_df.month == month) & (input_df.all_lice > 0)])
        output_dict[month].append(num_fish)

        num_infested_fish[month] = num_fish

    # get counts of all lice
    output_dict[lethality_type].append('All lice')
    for month in month_list:
        num_lice = input_df[(input_df.month == month)].all_lice.sum()
        output_dict[month].append(num_lice)

        num_all_lice[month] = num_lice

    # calculate prevalence
    output_dict[lethality_type].append('Prevalence')
    for month in month_list:
        prevalence = num_infested_fish[month] / num_all_fish[month]
        output_dict[month].append(prevalence)

    # calculate intensity
    output_dict[lethality_type].append('Intensity')
    for month in month_list:
        prevalence = num_all_lice[month] / num_infested_fish[month]
        output_dict[month].append(prevalence)

    # make into a dataframe and return
    output_df = pd.DataFrame(output_dict)

    # give the months friendly names
    output_df.rename(columns={3: 'Mar', 4: 'Apr', 5:'May', 6:'Jun', 7:'Jul'}, inplace=True)

    return output_df

### Non-lethal, 2008

In [10]:
non_lethal_2008_input_df = wild_df[(wild_df.sampling == NONLETHAL) & (wild_df.year == 2008)]
non_lethal_2008_df = fill_year_stats(non_lethal_2008_input_df, 'Non-lethal')

# write to file
non_lethal_2008_df.to_csv(OUTPUT_DIR / 'non_lethal_2008.csv', index=False)

non_lethal_2008_df.head(10)

,Non-lethal,Mar,Apr,May,Jun,Jul
0,All fish,29.000000,5669.000000,9451.000000,7997.000000,560.000000
1,Non-infested,25.000000,5216.000000,8160.000000,5978.000000,221.000000
2,Infested,4.000000,453.000000,1291.000000,2019.000000,339.000000
3,All lice,5.000000,504.000000,1738.000000,3114.000000,645.000000
4,Prevalence,0.137931,0.079908,0.136599,0.252470,0.605357
5,Intensity,1.250000,1.112583,1.346243,1.542348,1.902655


### Non-lethal, 2009

In [11]:
non_lethal_2009_input_df = wild_df[(wild_df.sampling == NONLETHAL) & (wild_df.year == 2009)]
non_lethal_2009_df = fill_year_stats(non_lethal_2009_input_df, 'Non-lethal')

# write to file
non_lethal_2009_df.to_csv(OUTPUT_DIR / 'non_lethal_2009.csv', index=False)

non_lethal_2009_df.head(10)


,Non-lethal,Apr,May,Jun,Jul
0,All fish,3174.000000,9929.000000,5495.000000,528.000000
1,Non-infested,3049.000000,8160.000000,4489.000000,396.000000
2,Infested,125.000000,1769.000000,1006.000000,132.000000
3,All lice,128.000000,2322.000000,1386.000000,187.000000
4,Prevalence,0.039382,0.178165,0.183076,0.250000
5,Intensity,1.024000,1.312606,1.377734,1.416667


### Lethal, 2008

In [12]:
lethal_2008_input_df = wild_df[(wild_df.sampling == LETHAL) & (wild_df.year == 2008)]
lethal_2008_df = fill_year_stats(lethal_2008_input_df, 'Lethal')

# write to file
lethal_2008_df.to_csv(OUTPUT_DIR / 'lethal_2008.csv', index=False)

lethal_2008_df.head(10)

,Lethal,Mar,Apr,May,Jun
0,All fish,544.000000,1432.000000,3348.000000,2343.000000
1,Non-infested,542.000000,1374.000000,3138.000000,2025.000000
2,Infested,2.000000,58.000000,210.000000,318.000000
3,All lice,2.000000,70.000000,302.000000,494.000000
4,Prevalence,0.003676,0.040503,0.062724,0.135723
5,Intensity,1.000000,1.206897,1.438095,1.553459


### Lethal, 2008

In [13]:
lethal_2009_input_df = wild_df[(wild_df.sampling != LETHAL) & (wild_df.year == 2009)]
lethal_2009_df = fill_year_stats(lethal_2009_input_df, 'Lethal')

# write to file
lethal_2009_df.to_csv(OUTPUT_DIR / 'lethal_2009.csv', index=False)

lethal_2009_df.head(10)

,Lethal,Apr,May,Jun,Jul
0,All fish,3174.000000,9929.000000,5495.000000,528.000000
1,Non-infested,3049.000000,8160.000000,4489.000000,396.000000
2,Infested,125.000000,1769.000000,1006.000000,132.000000
3,All lice,128.000000,2322.000000,1386.000000,187.000000
4,Prevalence,0.039382,0.178165,0.183076,0.250000
5,Intensity,1.024000,1.312606,1.377734,1.416667
